In [1]:
# Importar los módulos necesarios
import spacy
import json
import random
import os
import ast
from google.colab import drive
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy import displacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 26.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [27]:
drive.mount('/content/drive')
tfmPath = '/content/drive/MyDrive/TFM'
trainingFilesPath = tfmPath+'/archivos_entrenamiento'
validationFilePath = tfmPath+'/archivos_validacion'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
archivo = trainingFilesPath+'/masterfile_train.txt'

with open(archivo, 'r', encoding='utf-8') as file:
    content = file.read()
    data = ast.literal_eval(content)


In [64]:
def entrenar(datos, iteraciones=10):
    datos_gold = datos
    nlp = spacy.blank('es')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)
    for _, anns in datos_gold:
        for ent in anns.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.begin_training()
        for n in range(iteraciones):
            print("Iteración número " + str(n))
            random.shuffle(datos_gold)
            losses = {}
            for texto, anns in datos_gold:
              doc = nlp.make_doc(texto)
              ejemplo = Example.from_dict(doc, anns)
              nlp.update(
                  [ejemplo],
                  drop=0.2,
                  sgd=optimizer,
                  losses=losses)
            print(losses)
    return nlp

In [66]:
entrenamiento = entrenar(data)

Iteración número 0
{'ner': 3154.0068650455178}
Iteración número 1
{'ner': 338.7601325214446}
Iteración número 2
{'ner': 335.47340324927836}
Iteración número 3
{'ner': 233.13444652605472}
Iteración número 4
{'ner': 191.5168973518827}
Iteración número 5
{'ner': 230.1688736585444}
Iteración número 6
{'ner': 126.03031108187666}
Iteración número 7
{'ner': 166.89977833527828}
Iteración número 8
{'ner': 133.08529740296945}
Iteración número 9
{'ner': 124.8375920360674}


In [67]:
path_modelo = tfmPath+'/modelo'
entrenamiento.to_disk(path_modelo)

In [68]:
entrenamiento.to_disk("modelo")

In [69]:
nlp = spacy.load(path_modelo)

In [78]:
#Evaluación del modelo

def my_evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annotations in examples:
        pred = ner_model(input_)
        #print(pred,annotations)
        temp = Example.from_dict(pred, dict.fromkeys(annotations))
        example.append(temp)
    scores = scorer.score(example)
    return scores



In [79]:
archivo_validacion=validationFilePath+'/masterfile_val.txt'

with open(archivo_validacion, 'r', encoding='utf-8') as file:
    content = file.read()
    data_val = ast.literal_eval(content)


In [80]:
results = my_evaluate(nlp, data_val)
print(results)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None, 'cats_score': 0.0, 'cats_score_desc': 'macro F', 'cats_micro_p': 0.0, 'cats_micro_r': 0.0, 'cats_micro_f': 0.0, 'cats_macro_p': 0.0, 'cats_macro_r': 0.0, 'cats_macro_f': 0.0, 'cats_macro_auc': 0.0, 'cats_f_per_type': {}, 'cats_auc_per_type': {}}


In [75]:
texto_input = input("Por favor, introduce el texto:")


Por favor, introduce el texto:En el ámbito de la atención sanitaria, es fundamental brindar apoyo adecuado a quienes enfrentan diversas condiciones de salud. Por ejemplo, el discapacitado puede requerir acceso a terapias de rehabilitación especializadas, mientras que el minusválido a menudo necesita adaptaciones en su entorno para mejorar su movilidad y calidad de vida. De manera similar, el inválido puede beneficiarse de dispositivos de asistencia tecnológica que le permitan realizar tareas cotidianas de forma más independiente.  Existen condiciones más específicas, como la de un tetrapléjico, que generalmente requiere cuidados médicos continuos y soporte especializado para manejar su situación. Además, quienes padecen discapacidad a veces deben enfrentar barreras en el acceso a la educación y el empleo, lo cual limita sus oportunidades de desarrollo personal y profesional.  Por otro lado, algunos individuos que sufren discapacidad debido a accidentes o enfermedades pueden experimenta

In [76]:
doc = nlp(texto_input)
displacy.render(doc, style='ent', jupyter=True)

In [77]:
if len(doc.ents) == 0:
  print(f"No se han encontrado mejoras según los criterios establecidos.")
# Recorrer las entidades reconocidas
for ent in doc.ents:
    # Verificar la etiqueta de la entidad y actuar en consecuencia
    if ent.label_ == "alu":
        print(f"Considere cambiar {ent.text} por 'el alumnado'. ")
    if ent.label_ == "profs":
        print(f"Considere cambiar {ent.text} por 'el profesorado'. ")
    if ent.label_ == "inv":
        print(f"Considere cambiar {ent.text} por 'el equipo de investigación'. ")
    if ent.label_ == "hab":
      print(f"Considere cambiar {ent.text} por 'la población'. ")
    if ent.label_ == "cam":
      print(f"Considere cambiar {ent.text} por 'el personal de servicio'. ")
    if ent.label_ == "ing":
      print(f"Considere cambiar {ent.text} por 'el equipo de ingeniería'. ")
    if ent.label_ == "bus":
      print(f"Considere cambiar {ent.text} por 'las personas de negocios'. ")
    if ent.label_ == "cl":
      print(f"Considere cambiar {ent.text} por 'la clientela' ")
    if ent.label_ == "dis1" or ent.label_ == "dis2" or ent.label_ == "dis3" or ent.label_ == "dis4" or ent.label_ == "dis5":
      print(f"Considere cambiar {ent.text} por 'la persona con discapacidad'.")
    if ent.label_ == "dis6":
      print(f"Considere cambiar {ent.text} por 'la persona con tetrapléjia'.")
    if ent.label_ == "vdis1" or ent.label_ == "vdis2":
      print(f"Considere cambiar {ent.text} por 'tener discapacidad' ")

Considere cambiar discapacitado por 'la persona con discapacidad'.
Considere cambiar minusválido por 'la persona con discapacidad'.
Considere cambiar inválido por 'la persona con discapacidad'.
Considere cambiar tetrapléjico por 'la persona con tetrapléjia'.
Considere cambiar padecen discapacidad por 'tener discapacidad' 
Considere cambiar sufren discapacidad por 'tener discapacidad' 
Considere cambiar retrasado por 'la persona con discapacidad'.
